In [16]:
import boto3
import pandas as pd

In [2]:


s3 = boto3.client('s3')
s3.download_file('data-handling-public', 'products.csv',
                 'cred/products.csv')

In [17]:
aws_bucket ='data-handling-public'
s3_key= 'products.csv'
local_path= 'cred/products.csv'

In [44]:

response = s3.get_object(Bucket=aws_bucket, Key=s3_key)

status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 get_object response. Status - {status}")
    products_df = pd.read_csv(response.get("Body"))
    
else:
    print(f"Unsuccessful S3 get_object response. Status - {status}")

Successful S3 get_object response. Status - 200


In [ ]:
products_df

In [46]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1853 non-null   int64 
 1   product_name   1849 non-null   object
 2   product_price  1849 non-null   object
 3   weight         1849 non-null   object
 4   category       1849 non-null   object
 5   EAN            1849 non-null   object
 6   date_added     1849 non-null   object
 7   uuid           1849 non-null   object
 8   removed        1849 non-null   object
 9   product_code   1849 non-null   object
dtypes: int64(1), object(9)
memory usage: 144.9+ KB


In [29]:
import re




def convert_to_kg(weight):
    # Handle multiple units (e.g., "8 x 85g")
    multiple_match = re.match(r'(\d+)\s*x\s*(\d+\.?\d*)(\w+)', weight)
    if multiple_match:
        count = int(multiple_match.group(1))
        value = float(multiple_match.group(2))
        unit = multiple_match.group(3)
        weight_value = count * value
    else:
        # Extract value and unit
        match = re.match(r'(\d+\.?\d*)(\w+)', weight)
        if not match:
            return None
        weight_value = float(match.group(1))
        unit = match.group(2)

    # Conversion factors
    if unit == 'kg':
        return weight_value
    elif unit == 'g':
        return weight_value * 0.001
    elif unit == 'ml':
        return weight_value * 0.001  # Assuming 1:1 ratio for ml to g
    elif unit == 'oz':
        return weight_value * 0.0283495
    else:
        return None  # Unknown unit



In [47]:
products_df['date_added'] = pd.to_datetime(
    products_df['date_added'], format='%Y-%m-%d', errors='coerce')

In [48]:
products_df=products_df.dropna()

In [41]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1849 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1849 non-null   Int64 
 1   product_name   1849 non-null   string
 2   product_price  1849 non-null   string
 3   weight         1849 non-null   string
 4   category       1849 non-null   string
 5   EAN            1849 non-null   string
 6   date_added     1849 non-null   string
 7   uuid           1849 non-null   string
 8   removed        1849 non-null   string
 9   product_code   1849 non-null   string
dtypes: Int64(1), string(9)
memory usage: 160.7 KB


In [34]:
products_df = products_df.convert_dtypes()

In [35]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1853 non-null   Int64 
 1   product_name   1849 non-null   string
 2   product_price  1849 non-null   string
 3   weight         1849 non-null   string
 4   category       1849 non-null   string
 5   EAN            1849 non-null   string
 6   date_added     1849 non-null   string
 7   uuid           1849 non-null   string
 8   removed        1849 non-null   string
 9   product_code   1849 non-null   string
dtypes: Int64(1), string(9)
memory usage: 146.7 KB


In [49]:

# Apply conversion to the DataFrame
products_df['weight'] = products_df['weight'].apply(convert_to_kg)

In [ ]:
https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json

In [26]:
type(products_weights)

pandas.core.series.Series

In [51]:
import requests

In [55]:
header = {"x-api-key": "yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX"}

In [54]:
url ='https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'

In [57]:
def retrieve_data( header,url):
    try:
            response = requests.get(url, headers=header)
            # This will raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()

            # Process the response if no exceptions were raised
            data = response.json()  # Assuming the response is in JSON f

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")
    except ValueError as json_err:
        print(f"JSON decoding failed: {json_err}")

    return data

In [59]:
json_data = retrieve_data(header=header,url=url)